# Proyek Akhir Desain & Analisis Algoritma (DAA)
## Topik 9: Sudoku Solver 

**Kelompok 5 — Kelas Sains Data B**
**Program Studi Sains Data - FTISD UNS**

 Anggota Kelompok:
1. **Bagus Muhammad R.W.** (NIM: [NIM]) - *Project Manager*
2. **Michael christian S.G.** (NIM: [NIM]) - *Project Manager*
3. **Maulana Naufal Habibie** (NIM: L0224051) - *Lead Programmer*

## BAGIAN A: PENDAHULUAN & DEFINISI MASALAH

### 1.1 Latar Belakang & Motivasi
Sudoku adalah masalah pemenuhan kendala (*Constraint Satisfaction Problem*) yang termasuk dalam kelas kompleksitas **NP-Complete** pada grid $n \times n$. Algoritma pencarian dasar (*Brute Force Backtracking*) seringkali gagal menyelesaikan puzzle tingkat sulit dalam waktu yang wajar karena besarnya ruang pencarian (*search space*) yang harus dieksplorasi secara buta.

Untuk mengatasi hal ini, diperlukan teknik optimasi berupa **Heuristik (MRV)** atau **Pruning (Forward Checking)**. Proyek ini bertujuan untuk membuktikan secara empiris bagaimana teknik-teknik tersebut dapat memangkas pohon pencarian pada kasus-kasus ekstrem (Hard Instances).

### 1.2 Rumusan Masalah Formal
* **Input:** Matriks $9 \times 9$ (Instance Sudoku Set-B Hard) dengan sel kosong bernilai `0`.
* **Output:** Matriks $9 \times 9$ yang terisi penuh angka 1-9 tanpa melanggar aturan baris, kolom, dan blok.
* **Kendala Utama:** Algoritma harus mampu menyelesaikan puzzle dalam batas waktu (*timeout*) yang ditentukan.

### 1.3 Tujuan & Output Inti Proyek
Sesuai dengan spesifikasi tugas (Topik No. 9), proyek ini berfokus pada pembuktian efisiensi algoritma melalui metrik berikut:

1.  **Rasio Solusi (*Success Rate*):** Persentase puzzle yang berhasil diselesaikan sebelum *timeout*.
2.  **Median Waktu Eksekusi:** Menggunakan median (bukan rata-rata) untuk menghindari bias dari outlier waktu yang ekstrem.
3.  **Log Node (Bukti Pruning):** Mencatat jumlah node yang dikunjungi (*nodes explored*) untuk memvisualisasikan seberapa banyak cabang pohon pencarian yang berhasil dipangkas oleh algoritma.

Kami akan membandingkan:
* **Algoritma A:** Backtracking dengan **MRV** (*Minimum Remaining Values*).
* **Algoritma B:** Backtracking dengan **Forward Checking**.

---

### 1.4 Dataset (Instance Lock)
Eksperimen menggunakan dataset terkunci untuk memastikan konsistensi pengujian:
* **Nama Dataset:** `test_final_50.csv`
* **Karakteristik:** 50 Puzzle Sudoku Kategori **"Hard"** (Set-B).
* **Alasan Pemilihan:** Puzzle level sulit diperlukan untuk memperlihatkan *trade-off* dan manfaat nyata dari teknik pruning; pada puzzle mudah, perbedaan algoritma seringkali tidak terlihat signifikan.

In [1]:
import pandas as pd
import os

# Path ke dataset yang sudah dikunci
DATA_PATH = 'data/test_final_50.csv'

def load_data():
    if not os.path.exists(DATA_PATH):
        print(f"❌ File {DATA_PATH} tidak ditemukan.")
        return None
    
    df = pd.read_csv(DATA_PATH)
    print(f"✅ Dataset Termuat: {len(df)} puzzle 'Hard'")
    print("Contoh Data:")
    return df

df_sudoku = load_data()
if df_sudoku is not None:
    display(df_sudoku.head())

✅ Dataset Termuat: 50 puzzle 'Hard'
Contoh Data:


,id,puzzle,solution
0,1,..3.2.5....5.7.2...9.65.....3..........8....4....,8731295466154782932946538171397456827528619344...
1,2,......8.....9.......5.32.1.2...6...76............,4731568921269483759857326142483691576915872433...
2,3,...4..5...42....6.5....61..9.........561...4.....,1694325787428519635387961249172436853561897422...
3,4,..46...92......5..2............7...3..69...8.....,1746358926832195742957841364125789637569432819...
4,5,..31..5..6....9........4....8.....2.3.45..6......,8931675426452398717128549365896731243745216982...


## BAGIAN B: DESAIN ALGORITMA & PSEUDOCODE

Berikut adalah desain detail dari kedua algoritma. Pseudocode disajikan dalam gaya *Python-like* untuk memperjelas perbedaan logika implementasi, terutama pada letak "biaya komputasi" (apakah di awal pencarian atau saat pengisian nilai).

### 2.1 Algoritma A: Backtracking + MRV (*Minimum Remaining Values*)
**Logika Utama:**
Algoritma ini menggunakan strategi *Fail-First*. Perhatikan blok `[HEURISTIC START]` di bawah. Sebelum melakukan pengisian angka, algoritma melakukan *scanning* seluruh papan ($9 \times 9$) untuk mencari sel paling kritis (opsi tersedikit).

```python
FUNCTION Solve_MRV(board):
    # 1. Basis Rekursi: Cek apakah papan sudah penuh
    IF board is full:
        RETURN True

    # ====================================================
    # [HEURISTIC START] Mencari Variable Terbaik (MRV)
    # ====================================================
    best_row, best_col = None, None
    min_options = 10  # Inisialisasi angka > 9

    # Biaya Komputasi: Loop seluruh papan (O(N^2))
    FOR r from 0 to 8:
        FOR c from 0 to 8:
            IF board[r][c] is EMPTY:
                # Hitung berapa angka (1-9) yang valid di sini
                count = count_valid_candidates(board, r, c)
                
                IF count < min_options:
                    min_options = count
                    best_row, best_col = r, c
                    
                    # Optimasi: Jika opsi cuma 1, langsung pilih
                    IF min_options == 1: BREAK Loop
    
    # Jika ada sel kosong tapi min_options 0, berarti jalan buntu
    IF min_options == 0:
        RETURN False
    # ====================================================
    # [HEURISTIC END]
    # ====================================================

    # 2. Backtracking Standard pada sel terpilih
    FOR num from 1 to 9:
        IF is_safe(board, best_row, best_col, num):
            
            board[best_row][best_col] = num  # Pasang
            
            IF Solve_MRV(board) is True:     # Rekursi
                RETURN True
            
            board[best_row][best_col] = 0    # Backtrack (Reset)

    RETURN False
```

### 2.2 Algoritma B: Backtracking + Forward Checking
**Logika Utama:**
Algoritma ini menggunakan strategi *Look-Ahead Pruning*. Perhatikan blok `[PRUNING START]` di bawah. Algoritma ini "mahal" di memori karena harus menyalin (`deepcopy`) domain di setiap langkah untuk mengantisipasi *backtrack*.

```python
FUNCTION Solve_FC(board, domains):
    # 1. Basis Rekursi
    IF board is full:
        RETURN True

    # 2. Pilih Sel Kosong (Strategi Standar / First Available)
    row, col = find_first_empty_cell(board)

    # 3. Iterasi hanya pada angka yang tersedia di DOMAIN
    current_candidates = list(domains[row][col])

    FOR value IN current_candidates:
        
        # A. Pasang Angka
        board[row][col] = value
        
        # ====================================================
        # [PRUNING START] Forward Checking
        # ====================================================
        # Backup domain lama (Boros Memori tapi Wajib untuk Undo)
        domains_backup = deepcopy(domains)
        valid_future = True

        # Update tetangga (Baris, Kolom, Blok yang sama)
        neighbors = get_neighbors(row, col)
        
        FOR (nr, nc) IN neighbors:
            IF value IN domains[nr][nc]:
                remove value FROM domains[nr][nc]
                
                # PRUNING: Jika domain tetangga jadi kosong, Stop!
                IF domains[nr][nc] is EMPTY:
                    valid_future = False
                    BREAK Loop
        # ====================================================
        # [PRUNING END]
        # ====================================================

        # B. Rekursi (Hanya jika masa depan aman)
        IF valid_future is True:
            IF Solve_FC(board, domains) is True:
                RETURN True

        # C. Backtrack & Restore State
        board[row][col] = 0
        domains = domains_backup  # Restore domain

    RETURN False
```

### 2.3 Analisis Kompleksitas Teoretis

Analisis ini membandingkan kinerja teoretis Algoritma A (MRV) dan Algoritma B (Forward Checking) berdasarkan struktur pohon pencarian (*Search Tree*) dan penggunaan memori.

**Notasi:**
* $N$: Jumlah total sel dalam papan Sudoku ($N = 81$).
* $m$: Jumlah sel kosong awal (kedalaman maksimum pohon rekursi).
* $d$: Ukuran domain (maksimal 9).

#### A. Tabel Perbandingan Kompleksitas

| Aspek | Algoritma A (Backtracking + MRV) | Algoritma B (Forward Checking) |
| :--- | :--- | :--- |
| **Kompleksitas Waktu (Worst Case)** | $O(9^m)$ | $O(9^m)$ |
| **Kompleksitas Waktu (Average)** | **Sangat Cepat** (Branching factor kecil) | **Cepat** (Pruning efektif) |
| **Kompleksitas Ruang (Space)** | $O(m)$ (Efisien) | $O(m \times N)$ (Boros Memori) |
| **Lokasi *Overhead*** | Awal node (Scanning Papan) | Dalam node (Update & Copy Domain) |

#### B. Penjelasan Detail

**1. Analisis Kompleksitas Waktu (*Time Complexity*)**
* **Worst Case:** Pada kasus terburuk (misalnya puzzle kosong atau puzzle yang dirancang anti-heuristik), kedua algoritma tetap memiliki kompleksitas eksponensial $O(d^m) \approx O(9^m)$. Hal ini karena keduanya berbasis DFS yang harus menelusuri seluruh kemungkinan kombinasi jika tidak ada solusi.
* **Average Case (Efisiensi Praktis):**
    * **MRV:** Mengurangi *effective branching factor* ($b$). Dengan selalu memilih sel yang opsinya paling sedikit (misal sisa 1 atau 2), pohon pencarian menjadi "kurus" dan dalam, menghindari percabangan yang tidak perlu di level atas.
    * **Forward Checking:** Mengurangi ukuran ruang pencarian dengan mekanisme *pruning*. Cabang yang mengarah ke jalan buntu dipotong lebih awal sebelum algoritma masuk terlalu dalam.

**2. Analisis Kompleksitas Ruang (*Space Complexity*)**
* **Algoritma A (MRV): $O(m)$**
    Hanya membutuhkan memori untuk tumpukan rekursi (*recursion stack*) sedalam $m$. Tidak ada struktur data tambahan yang disimpan antar-rekursi. Ini sangat hemat memori.
* **Algoritma B (Forward Checking): $O(m \times N)$**
    Di setiap tingkat kedalaman rekursi, algoritma harus menyimpan salinan (*snapshot*) dari tabel Domain seluruh papan ($9 \times 9$) untuk keperluan pemulihan (*backtracking*).
    * *Biaya:* Jika kedalaman rekursi mencapai 50 level, maka program menyimpan 50 salinan papan di memori. Ini adalah *trade-off* utama dari algoritma ini.

**3. Kesimpulan Trade-off**
* **Algoritma MRV** menukar waktu CPU (untuk scanning papan mencari minimum) demi mendapatkan urutan eksekusi terbaik.
* **Algoritma Forward Checking** menukar Memori (untuk menyimpan domain) demi memangkas cabang pohon pencarian.

## BAGIAN C: METODOLOGI EKSPERIMEN

### 3.1 Protokol Pengujian
[cite_start]Untuk memastikan hasil yang adil dan *reproducible* sesuai checklist mingguan,eksperimen dilakukan dengan protokol berikut:

1.  **Lingkungan Eksekusi:**
    * **Hardware:** 13th Gen Intel(R) Core(TM) i7-13650HX (20 CPUs), ~2.6GHz, RAM 20GB ddr5
    * **Software:** Python 3.12.6 
    * **Timeout:** Batas waktu pencarian adalah **60 detik** per puzzle.

2.  **Skenario Uji:**
    * Dataset: 50 Puzzle "Hard" (`test_final_50.csv`).
    * **Repeats:** Setiap algoritma dijalankan **10 kali** per puzzle untuk mendapatkan waktu rata-rata yang stabil.

### 3.2 Metrik Evaluasi (Output Inti)
[cite_start]Sesuai spesifikasi proyek Topik 9, kinerja diukur menggunakan:

1.  **Rasio Solusi (*Success Rate*):** Persentase puzzle yang berhasil diselesaikan sebelum *timeout*.
2.  **Median Waktu (*Median Time*):** Median waktu eksekusi (ms) pada kasus sukses.
3.  **Jumlah Node (*Nodes Explored*):** Total pemanggilan rekursif sebagai bukti efektivitas *pruning*.

In [5]:
import copy
import time
import sys

# Meningkatkan batas rekursi jaga-jaga jika puzzle sangat dalam
sys.setrecursionlimit(10000)

class SudokuSolver:
    def __init__(self):
        self.nodes_explored = 0  # Counter untuk menghitung jumlah langkah (bukti pruning)

    # ==========================================
    # FUNGSI BANTU (HELPER)
    # ==========================================
    def is_safe(self, board, row, col, num):
        """Mengecek apakah angka 'num' aman ditaruh di (row, col) sesuai aturan Sudoku."""
        # Cek Baris & Kolom
        for x in range(9):
            if board[row][x] == num or board[x][col] == num:
                return False
        
        # Cek Blok 3x3
        start_row, start_col = row - row % 3, col - col % 3
        for i in range(3):
            for j in range(3):
                if board[i + start_row][j + start_col] == num:
                    return False
        return True

    def find_empty_standard(self, board):
        """Mencari sel kosong pertama (urutan kiri-atas ke kanan-bawah)."""
        for i in range(9):
            for j in range(9):
                if board[i][j] == 0:
                    return (i, j)
        return None

    # ==========================================
    # ALGORITMA A: Backtracking + MRV
    # ==========================================
    def solve_mrv(self, board):
        """
        Implementasi Backtracking dengan Heuristik MRV (Minimum Remaining Values).
        """
        # 1. HEURISTIK: Cari sel kosong dengan opsi paling sedikit
        empty_pos = self._find_mrv_cell(board)
        
        # Basis: Jika tidak ada sel kosong, puzzle selesai
        if not empty_pos:
            return True
            
        row, col = empty_pos
        
        # 2. Iterasi Angka 1-9
        for num in range(1, 10):
            if self.is_safe(board, row, col, num):
                self.nodes_explored += 1  # Catat node yang dikunjungi
                
                board[row][col] = num
                
                # Rekursi
                if self.solve_mrv(board):
                    return True
                    
                board[row][col] = 0  # Backtrack (Reset)
                
        return False

    def _find_mrv_cell(self, board):
        """Helper khusus MRV: Scan seluruh papan cari domain terkecil."""
        min_options = 10
        best_cell = None
        
        for r in range(9):
            for c in range(9):
                if board[r][c] == 0:
                    options_count = 0
                    for num in range(1, 10):
                        if self.is_safe(board, r, c, num):
                            options_count += 1
                    
                    if options_count < min_options:
                        min_options = options_count
                        best_cell = (r, c)
                        if min_options == 1: return best_cell # Optimasi
        return best_cell

    # ==========================================
    # ALGORITMA B: Backtracking + Forward Checking
    # ==========================================
    def solve_fc(self, board, domains=None):
        """
        Implementasi Backtracking dengan Forward Checking (Pruning).
        """
        # Inisialisasi domain di langkah pertama
        if domains is None:
            domains = self._get_initial_domains(board)
            
        # Basis: Jika papan penuh (tidak ada sel kosong)
        if self.find_empty_standard(board) is None:
            return True

        # 1. Pilih sel kosong (Urutan Standar)
        row, col = self.find_empty_standard(board)
        
        # 2. Iterasi hanya angka yang ada di DOMAIN saat ini
        current_options = list(domains[row][col])
        
        for val in current_options:
            self.nodes_explored += 1
            
            # A. Pasang Angka
            board[row][col] = val
            
            # B. FORWARD CHECKING (Pruning)
            domains_backup = copy.deepcopy(domains) # Backup state (Boros Memori tapi wajib)
            
            # Update domain tetangga. Jika aman (tidak ada domain kosong), lanjut rekursi
            if self._update_neighbors_fc(domains, row, col, val):
                if self.solve_fc(board, domains):
                    return True
            
            # C. Backtrack
            board[row][col] = 0
            domains = domains_backup # Restore state (Undo perubahan domain)
            
        return False

    def _get_initial_domains(self, board):
        """Helper FC: Membangun domain awal untuk setiap sel kosong."""
        domains = [[set() for _ in range(9)] for _ in range(9)]
        for r in range(9):
            for c in range(9):
                if board[r][c] == 0:
                    for num in range(1, 10):
                        if self.is_safe(board, r, c, num):
                            domains[r][c].add(num)
        return domains

    def _update_neighbors_fc(self, domains, row, col, val):
        """Helper FC: Menghapus 'val' dari domain tetangga. Return False jika fail."""
        # Tetangga: Baris, Kolom, Blok
        neighbors = set()
        for i in range(9): neighbors.add((row, i))
        for i in range(9): neighbors.add((i, col))
        
        sr, sc = row - row % 3, col - col % 3
        for i in range(3):
            for j in range(3):
                neighbors.add((sr + i, sc + j))
        
        for (r, c) in neighbors:
            if (r, c) != (row, col): # Jangan hapus domain diri sendiri
                if val in domains[r][c]:
                    domains[r][c].remove(val)
                    if len(domains[r][c]) == 0: return False # Pruning terjadi (Jalan buntu)
        return True

In [7]:
# --- SANITY CHECK (UJI COBA & PARSING DATA) ---

def parse_puzzle(puzzle_str):
    """
    Mengubah string puzzle menjadi matriks 9x9.
    KHUSUS: Menangani simbol '.' sebagai 0 (sel kosong).
    """
    grid = []
    for i in range(9):
        row = []
        for j in range(9):
            # Ambil karakter dari string (index baris * 9 + kolom)
            char = puzzle_str[i * 9 + j]
            
            # Jika karakter adalah '.', ubah jadi 0. 
            # Jika angka, ubah jadi integer.
            if char == '.':
                row.append(0)
            elif char.isdigit():
                row.append(int(char))
            else:
                # Jaga-jaga jika ada karakter aneh lain, anggap 0
                row.append(0)
        grid.append(row)
    return grid

# Ambil 1 contoh puzzle dari dataset (Pastikan Cell Load Data sudah dijalankan)
try:
    # Mengambil baris pertama dari dataframe
    sample_data = df_sudoku.iloc[0]
    sample_str = sample_data['puzzle']
    sample_id = sample_data['id'] # Menggunakan ID jika ada kolom ID

    print(f"🧩 Menguji Puzzle ID {sample_id}")
    print(f"   String Input: {sample_str[:15]}... (Menggunakan titik '.')")
    
    # --- TEST 1: ALGORITMA MRV ---
    solver = SudokuSolver()
    board_mrv = parse_puzzle(sample_str) # Parsing . jadi 0
    
    print("\n[1] Menjalankan Algoritma A (MRV)...")
    start = time.time()
    if solver.solve_mrv(board_mrv):
        dur = (time.time() - start) * 1000
        print(f"   ✅ Sukses! Waktu: {dur:.2f} ms")
        print(f"   📉 Nodes Explored: {solver.nodes_explored}")
    else:
        print("   ❌ Gagal menemukan solusi.")

    # --- TEST 2: ALGORITMA FORWARD CHECKING ---
    solver = SudokuSolver()
    board_fc = parse_puzzle(sample_str) # Reset papan baru
    
    print("\n[2] Menjalankan Algoritma B (Forward Checking)...")
    start = time.time()
    if solver.solve_fc(board_fc):
        dur = (time.time() - start) * 1000
        print(f"   ✅ Sukses! Waktu: {dur:.2f} ms")
        print(f"   📉 Nodes Explored: {solver.nodes_explored}")
        print("   (Perhatikan perbedaan Nodes Explored dengan MRV)")
    else:
        print("   ❌ Gagal menemukan solusi.")

except NameError:
    print("⚠️ Error: Variabel 'df_sudoku' tidak ditemukan. Jalankan Cell 'Load Data' (Bagian A) dulu.")
except Exception as e:
    print(f"⚠️ Error Lain: {e}")

🧩 Menguji Puzzle ID 1
   String Input: ..3.2.5....5.7.... (Menggunakan titik '.')

[1] Menjalankan Algoritma A (MRV)...
   ✅ Sukses! Waktu: 2099.07 ms
   📉 Nodes Explored: 18746

[2] Menjalankan Algoritma B (Forward Checking)...
   ✅ Sukses! Waktu: 1083819.08 ms
   📉 Nodes Explored: 4170890
   (Perhatikan perbedaan Nodes Explored dengan MRV)
